<a href="https://colab.research.google.com/github/pogpog/sentiment-analysis/blob/main/notebooks/01-dataset-finetune/dataset_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets huggingface_hub

In [ ]:
from datasets import load_dataset, Dataset, ClassLabel, Features, Value
import pandas as pd

df = pd.read_csv('data.csv')
df.head()

,Text,Intent,Sentiment
0,No my current balance as of 29/06/23 is Â£118.88,Account Details,neutral
1,According to me we're up to date,Account Details,neutral
2,There was linkage which landlord just fixed,Account Details,neutral
3,Hearing some false info from them.,Account Details,negative
4,like to know how much,Account Enquiry,neutral


In [ ]:
if 'Intent' in df.columns:
    df.drop('Intent', axis=1, inplace=True)

df.head()

if 'Sentiment' in df.columns:
    df.rename(columns={'Sentiment': 'labels'}, inplace=True)

if 'Text' in df.columns:
    df.rename(columns={'Text': 'text'}, inplace=True)
df.head()

,text,labels
0,No my current balance as of 29/06/23 is Â£118.88,neutral
1,According to me we're up to date,neutral
2,There was linkage which landlord just fixed,neutral
3,Hearing some false info from them.,negative
4,like to know how much,neutral


In [ ]:

label_counts = df['labels'].value_counts()
print(label_counts)

labels
neutral     107
negative     83
positive     48
anger        12
Name: count, dtype: int64


In [ ]:
# prompt: Using dataframe df: pie chart

import altair as alt

def pie_chart(df, label_counts):
    # Count the occurrences of each label
    if label_counts is None:
        label_counts = df['labels'].value_counts().reset_index()
    label_counts.columns = ['labels', 'count']

    # Create the pie chart
    chart = alt.Chart(label_counts).mark_arc().encode(
        theta='count',
        color='labels',
        tooltip=['labels', 'count']
    ).display()

pie_chart(df, label_counts=None)
label_counts = df['labels'].value_counts()
print(label_counts)

alt.Chart(...)

labels
neutral     107
negative     83
positive     48
anger        12
Name: count, dtype: int64


In [ ]:
df = df[df.labels != 'joy']

In [ ]:
# Get all rows where the "labels" column is in the top 10 by count
top_10_labels = df['labels'].value_counts().nlargest(10).index
df = df[df['labels'].isin(top_10_labels)]

pie_chart(df, label_counts)

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
dataset = dataset.class_encode_column("labels")

Casting to class labels:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
dataset.features

{'text': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['anger', 'negative', 'neutral', 'positive'], id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
label_names = dataset.unique('labels')
print(label_names)

[2, 1, 3, 0]


In [ ]:
class_label = ClassLabel(names=label_names)

In [ ]:
# Create dataset with correct features
features = Features({
    "text": Value("string"),
    "labels": class_label  # Store mapping in ClassLabel
})

In [ ]:
print(dataset[1])

{'text': "According to me we're up to date", 'labels': 2, '__index_level_0__': 1}


In [ ]:
data_name = 'data-sentiments-01'

In [ ]:
dataset.save_to_disk(data_name)

Saving the dataset (0/1 shards):   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
#intents-test-03 Optionally push to HuggingFace
if True:
    from huggingface_hub import notebook_login
    notebook_login()
    dataset.push_to_hub(f'pogpog/{data_name}')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
data_name

'data-sentiments-01'